In [1]:
import os, shutil, subprocess
import gromacs
from htmd.ui import *
import openbabel

gromacs.config.setup()

# os.system('source /dls/science/groups/i04-1/software/gromacs/bin/GMXRC')

ffevaluate module is in beta version


/dls/science/groups/i04-1/software/anaconda/envs/docking/lib/python3.6/site-packages/pyemma/util/log.py:65: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  D = yaml.load(f2)



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/

You are on the latest HTMD version (1.15.2).



In [2]:
def prep_protein(protein_pdb, keep='protein'):
    mol = Molecule(protein_pdb)
    mol.filter(keep)
    prot = proteinPrepare(mol)
    preppdb = protein_pdb.replace('.pdb', '_prepared.pdb')
    prot.write(preppdb)

    return prot, preppdb


def genion(s, o, p, pname='NA', nname='CL', neutral=True, option=13):
    command = f"echo {option} | gmx genion -s {s} -o {o} -p {p} -pname {pname} -nname {nname}"
    if neutral:
        command += ' -neutral'
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    out, err = process.communicate()
    
    if 'Error' in out.decode('ascii') or 'Error' in err.decode('ascii'):
        raise Exception('Error:\n' + out.decode(ascii) + '\n' + err.decode(ascii))
    
    return out, err



In [4]:
# os.mkdir('20190806-md-3')
# shutil.copy(
#     '/dls/labxchem/data/2017/lb18145-1/processing/fragalysis/XX02KALRNA/XX02KALRNA-x1376_1/XX02KALRNA-x1376_1_apo.pdb',
# '20190806-md-3/')
# os.chdir('20190806-md-3')

In [29]:
# prot, prot_file = prep_protein('XX02KALRNA-x1376_1_apo.pdb')
gromacs.pdb2gmx(f=prot_file, o='XX02KALRNA.gro', p='XX02KALRNA.top', ff='amber99sb-star-ildnp', water='tip4p', ignh=True)

(0, None, None)

In [30]:
gromacs.editconf(f='XX02KALRNA.gro', o='XX02KALRNA_boxed.gro', bt='cubic', c=True, d=1)

(0, None, None)

In [31]:
gromacs.solvate(cp='XX02KALRNA_boxed.gro', cs='tip4p.gro', o='XX02KALRNA_solv.gro', p='XX02KALRNA.top')

(0, None, None)

In [32]:
with open('ions.mdp', 'w') as f:
    f.write(ions_mdp)
    
gromacs.grompp(f='ions.mdp', c='XX02KALRNA_solv.gro', p='XX02KALRNA.top', o='ions.tpr')

(0, None, None)

In [33]:
genion(s='ions.tpr', o='XX02KALRNA_solv_ions.gro', p='XX02KALRNA.top', pname='NA', nname='CL', 
       neutral=True, option=13)


(b"Will try to add 10 NA ions and 0 CL ions.\nSelect a continuous group of solvent molecules\nSelected 13: 'SOL'\n\nProcessing topology\nReplacing 10 solute molecules in topology file (XX02KALRNA.top)  by 10 NA and 0 CL ions.\n",
 b'                  :-) GROMACS - gmx genion, VERSION 5.1.4 (-:\n\n                            GROMACS is written by:\n     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   \n Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch \n  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen\n Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  \n    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff \n   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   \n   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  \n   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   \n                      

In [46]:
with open('emin1.mdp', 'w') as f:
    f.write(minim_mdp_1)
    
with open('emin2.mdp', 'w') as f:
    f.write(minim_mdp_2)
    
with open('emin3.mdp', 'w') as f:
    f.write(minim_mdp_3)
    
gromacs.grompp(f='emin1.mdp', c='XX02KALRNA_solv_ions.gro', p='XX02KALRNA.top', o='emin1.tpr')
gromacs.mdrun(v=True, deffnm='emin1')

gromacs.grompp(f='emin2.mdp', c='emin1.gro', p='XX02KALRNA.top', o='emin2.tpr')
gromacs.mdrun(v=True, deffnm='emin2')

gromacs.grompp(f='emin3.mdp', c='emin2.gro', p='XX02KALRNA.top', o='emin3.tpr')
gromacs.mdrun(v=True, deffnm='emin3')

(0, None, None)

In [47]:
with open('nvt.mdp', 'w') as f:
    f.write(nvt_mdp)
    
with open('npt.mdp', 'w') as f:
    f.write(npt_mdp)
    
gromacs.grompp(f='nvt.mdp', c='emin3.gro', p='XX02KALRNA.top', o='nvt.tpr')
gromacs.mdrun(v=True, deffnm='nvt')

gromacs.grompp(f='npt.mdp', c='nvt.gro', p='XX02KALRNA.top', o='npt.tpr')
gromacs.mdrun(v=True, deffnm='npt')

(0, None, None)

In [ ]:
with open('md.mdp', 'w') as f:
    f.write(md_mdp)
    
gromacs.grompp(f='md.mdp', c='npt.gro', p='XX02KALRNA.top', o='md.tpr')
gromacs.mdrun(v=True, deffnm='md')


In [65]:
opts = """
; NEIGHBORSEARCHING PARAMETERS
nstlist                  = 5
ns-type                  = Grid
pbc                      = xyz
rlist                    = 1.0

; OPTIONS FOR ELECTROSTATICS AND VDW
coulombtype              = PME
pme_order                = 4            ; cubic interpolation
fourierspacing           = 0.16         ; grid spacing for FFT
rcoulomb                 = 1.0
vdw-type                 = Cut-off
rvdw                     = 1.0
"""


ions_mdp = """
; ions.mdp - used as input into grompp to generate ions.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform
disre = simple
""" + opts

minim_mdp_1 = """
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 5000          ; Maximum number of (minimization) steps to perform
simulation-part = 1         ; simulation part for multiple parts
constraints = all-bonds
"""  + opts

minim_mdp_2 = """
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 5000          ; Maximum number of (minimization) steps to perform
simulation-part = 1         ; simulation part for multiple parts
constraints = h-bonds
"""  + opts

minim_mdp_3 = """
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 20000         ; Maximum number of (minimization) steps to perform
simulation-part = 1         ; simulation part for multiple parts
constraints = none
"""  + opts

nvt_mdp = """
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = 50000     ; 2 * 50000 = 100 ps
dt                      = 0.001     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = no        ; first dynamics run
constraint_algorithm    = lincs     ; holonomic constraints
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is off
pcoupl                  = no        ; no pressure coupling in NVT
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = yes       ; assign velocities from Maxwell distribution
gen_temp                = 300       ; temperature for Maxwell distribution
gen_seed                = -1        ; generate a random seed
"""

npt_mdp = """
define                  = -DPOSRES  ; position restrain the protein
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = 50000     ; 2 * 50000 = 100 ps
dt                      = 0.001     ; 2 fs
; Output control
nstxout                 = 500       ; save coordinates every 1.0 ps
nstvout                 = 500       ; save velocities every 1.0 ps
nstenergy               = 500       ; save energies every 1.0 ps
nstlog                  = 500       ; update log file every 1.0 ps
; Bond parameters
continuation            = yes       ; Restarting after NVT
constraint_algorithm    = lincs     ; holonomic constraints
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Nonbonded settings
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
refcoord_scaling        = com
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Velocity generation
gen_vel                 = no        ; Velocity generation is off
"""

md_mdp = """
; Run parameters
integrator              = md        ; leap-frog integrator
nsteps                  = 20000000    ; 2 * 500000 = 1000 ps (1 ns)
dt                      = 0.001     ; 1 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
compressed-x-grps       = System    ; save the whole system

; Bond parameters
continuation            = yes       ; Restarting after NPT
constraint_algorithm    = shake     ; holonomic constraints
shake-tol = 0.0001
constraints = none

; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)

; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT

; Temperature coupling is on
tcoupl                  = v-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K

; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1

; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off
"""

In [33]:
ls

posre_Protein_chain_A.itp           #XX02KALRNA_Protein_chain_A.itp.3#
#posre_Protein_chain_A.itp.1#       XX02KALRNA_Protein_chain_B.itp
#posre_Protein_chain_A.itp.2#       #XX02KALRNA_Protein_chain_B.itp.1#
#posre_Protein_chain_A.itp.3#       #XX02KALRNA_Protein_chain_B.itp.2#
posre_Protein_chain_B.itp           #XX02KALRNA_Protein_chain_B.itp.3#
#posre_Protein_chain_B.itp.1#       XX02KALRNA.top
#posre_Protein_chain_B.itp.2#       #XX02KALRNA.top.1#
#posre_Protein_chain_B.itp.3#       #XX02KALRNA.top.2#
XX02KALRNA_boxed.gro                #XX02KALRNA.top.3#
XX02KALRNA.gro                      #XX02KALRNA.top.4#
#XX02KALRNA.gro.1#                  #XX02KALRNA.top.5#
#XX02KALRNA.gro.2#                  #XX02KALRNA.top.6#
XX02KALRNA_Protein_chain_A.itp      XX02KALRNA-x1376_1_apo.pdb
#XX02KALRNA_Protein_chain_A.itp.1#  XX02KALRNA-x1376_1_apo_prepared.pdb
#XX02KALRNA_Protein_chain_A.itp.2#  XX02KALRNA-x1376_1_apo_prepared.pdbqt
